In [1]:
import preonpy
import numpy as np

from agent.model import Actor, Critic
from env.preon_env import *
from utils.util import *
from utils.options import Options
from copy import deepcopy
from utils.memory import ReplayMemory

In [3]:
float(np.random.randint(0,400))

212.0

In [2]:
opt = Options()

In [14]:
memory = ReplayMemory(100)

In [15]:
memory.push([0,0], [0,0], 0, [1,1], 0, False)
memory.push([1,1], [0,0], 0, [2,2], 0, False)
memory.push([2,2], [0,0], 0, [3,3], 0, False)

In [16]:
len(memory.memory)

3

In [16]:
memory.sample(2)

[Transition(state=[1, 1], goal=[0, 0], action=0, next_state=[2, 2], reward=0, terminal=False),
 Transition(state=[2, 2], goal=[0, 0], action=0, next_state=[3, 3], reward=0, terminal=False)]

In [21]:
t = memory.remove_first()
print(t)

None


In [7]:
env = preon_env("scene1.prscene")
state, info = env.reset()

AttributeError: 'str' object has no attribute 'path'

In [6]:
env = preon_env(opt.env_params)
state, _ = deepcopy(env.reset())

Started simulation from frame 0 to frame 2 with 8 thread(s).
Done simulating  2  frames.


In [7]:
state

(0.0, 12.0, 0.0, 0.0, 0.0)

In [3]:
print(state,info)

((0.0, 12.0, 0.0, 0.0, 0.0), (468.0, 468.0, 468.0))


In [4]:
actor = Actor(5, 3, 2)
critic = Critic(5, 3, 2)

In [5]:
goal = [100.0, 0.0]    # Poured volume, spillage

In [6]:
action = actor(to_tensor(np.array([state])),to_tensor(np.array([goal])))
#action = to_numpy(action).squeeze(0)
action = to_numpy(action)
print(action)

[[ 0.18515317 -0.97661507  0.96062768]]


In [7]:
values = critic(to_tensor(np.array([state])),to_tensor(np.array([goal])),to_tensor(action))
print(values)

Variable containing:
-2.4010 -1.2800 -0.2121
[torch.FloatTensor of size 1x3]



In [17]:
np.random.normal((0.5,-0.7,0.3),0.05)

array([ 0.52138292, -0.65525076,  0.35658651])

In [21]:
np.random.random_sample()

0.3662387990162894

In [35]:
state = [0,0,15,10,50]
goal = [50,50]
a= np.absolute(np.array(state[3:5]) - np.array(goal)) - 10
a

array([ 30, -10])

In [36]:
np.maximum(a,0)

array([30,  0])